In [1]:
import json
import pymongo

In [3]:
f = open('students.json')
data = json.load(f)

In [4]:
client = pymongo.MongoClient('mongodb://127.0.0.1:27017/')

In [5]:
mydb = client['student_db_and_collection']

In [6]:
collection = mydb.full_data

In [7]:
if isinstance(data,list):
    collection.insert_many(data)
else:
    collection.insert_one(data)

In [8]:
collection1 = mydb.q1
collection2 = mydb.q2
collection3 = mydb.q3
collection41 = mydb.q4c1
collection42 = mydb.q4c2
collection5 = mydb.q5
collection6 = mydb.q6
collection7 = mydb.q7

# 1) Find the student name who scored maximum scores in all (exam, quiz and homework)?

In [9]:
for i in collection.aggregate([{'$project' : {'name' : 1, 'Maximumofscores' : {'$max' : '$scores.score'}}},
                               {'$sort' : {'Maximumofscores' : -1}},
                               {'$limit' : 1}]) :
    collection1.insert_one(i)

# 2) Find students who scored below average in the exam and pass mark is 40%?

In [10]:
for i in collection.aggregate([{'$unwind' : '$scores'},
                               
                               {'$match': { 'scores.type': 'exam'}},
                               
                               {'$group': { '_id': 'null', 'avg_exam': { '$avg': '$scores.score'},
                               'examscorebelowaverageandabove40' : {'$addToSet' : {'_id' : '$_id', 'name' : '$name', 'examscore' :'$scores.score'}}}},
                               
                               {'$project' : {'_id' : 0,
                                            'examscore' : {
                                                         '$filter' : {
                                                          'input' : "$examscore",
                                                          'as' : "examscores",
                                                          'cond' : {
                                                                '$lt' : ["$$examscores.score", "$avg_exam"] }}}}},
                               {'$unwind' : '$examscore'},
                               {'$match' : {'examscore.score' : {'$gt' : 40}}}
                              ]):
    collection2.insert_one(i)

# 3) Find students who scored below pass mark and assign them as fail, and above pass mark as pass in all the categories.

In [12]:
for i in collection.aggregate([{'$project' : {'name' : 1, 'examcolumn' : {'$slice' : ['$scores', 1]},
                                              'quizcolumn' : {'$slice' : ['$scores', 1, 1]},
                                              'homeworkcolumn' : {'$slice' : ['$scores', 2, 1]}}},
                               {'$project' : {'name' : 1, 'examscore' : {'$min' : '$examcolumn.score'},
                                              'quizscore' : {'$min' : '$quizcolumn.score'},
                                              'homeworkscore' : {'$min' : '$homeworkcolumn.score'}}},
                               {'$group' : {'_id' : 'null', 'sumexam' : {'$sum' : '$examscore'}, 
                                           'sumquiz' : {'$sum' : '$quizscore'}, 'sumhomework' : {'$sum' : '$homeworkscore'}}},
                               {'$project' : {'_id' : 0, 'totalofexam' : '$sumexam', 'totalofquiz' : '$sumquiz',
                                             'totalofhomework' : '$sumhomework'}}
                               ]):
    collection41.insert_one(i)

for i in collection.aggregate([{'$project' : {'name' : 1, 'examcolumn' : {'$slice' : ['$scores', 1]},
                                              'quizcolumn' : {'$slice' : ['$scores', 1, 1]},
                                              'homeworkcolumn' : {'$slice' : ['$scores', 2, 1]}}},
                               {'$project' : {'name' : 1, 'examscore' : {'$min' : '$examcolumn.score'},
                                              'quizscore' : {'$min' : '$quizcolumn.score'},
                                              'homeworkscore' : {'$min' : '$homeworkcolumn.score'}}},
                               {'$group' : {'_id' : 'null', 'avgexam' : {'$avg' : '$examscore'}, 
                                           'avgquiz' : {'$avg' : '$quizscore'}, 'avghomework' : {'$avg' : '$homeworkscore'}}},
                               {'$project' : {'_id' : 0, 'averageofexam' : '$avgexam', 'averageofquiz' : '$avgquiz',
                                             'averageofhomework' : '$avghomework'}}
                               ]):
    collection42.insert_one(i)

# 5) Create a new collection which consists of students who scored below average in all the categories.

In [13]:
for i in collection.aggregate([{'$project' : {'name' : 1, 'examcolumn' : {'$slice' : ['$scores', 1]},
                                              'quizcolumn' : {'$slice' : ['$scores', 1, 1]},
                                              'homeworkcolumn' : {'$slice' : ['$scores', 2, 1]}}},
                               {'$project' : {'name' : 1, 'examscore' : {'$min' : '$examcolumn.score'},
                                              'quizscore' : {'$min' : '$quizcolumn.score'},
                                              'homeworkscore' : {'$min' : '$homeworkcolumn.score'}}},
                               {'$group' : {'_id' : 'null', 'avgexam' : {'$avg' : '$examscore'}, 
                                           'avgquiz' : {'$avg' : '$quizscore'}, 'avghomework' : {'$avg' : '$homeworkscore'},
                                           'allscore' : {'$addToSet' : {'_id' : '$_id', 'name' : '$name', 'examscore' :'$examscore',
                                                                       'quizscore' : '$quizscore', 'homeworkscore' : '$homeworkscore'}}}},
                               {'$project' : {'_id' : 0, 'avgquiz' : '$avgquiz', 'avghomework' : '$avghomework',
                                            'allscore' : {
                                                         '$filter' : {
                                                          'input' : "$allscore",
                                                          'as' : "allscores",
                                                          'cond' :{'$lt' : ["$$allscores.examscore", "$avgexam"]
                                                           }}}}},
                               {'$project' : {'_id' : 0, 'avghomework' : '$avghomework',
                                            'allscore' : {
                                                         '$filter' : {
                                                          'input' : "$allscore",
                                                          'as' : "allscores",
                                                          'cond' :{'$lt' : ["$$allscores.quizscore", "$avgquiz"]
                                                           }}}}},
                               {'$project' : {'_id' : 0,
                                            'allscoresbelowaverage' : {
                                                         '$filter' : {
                                                          'input' : "$allscore",
                                                          'as' : "allscores",
                                                          'cond' :{'$lt' : ["$$allscores.homeworkscore", "$avghomework"]
                                                           }}}}}
                               ]):
    collection5.insert_one(i)

# 6) Create a new collection which consists of students who scored below the fail mark in all the categories.

In [14]:
for i in collection.aggregate([{'$project' : {'name' : 1, 'examcolumn' : {'$slice' : ['$scores', 1]},
                                              'quizcolumn' : {'$slice' : ['$scores', 1, 1]},
                                              'homeworkcolumn' : {'$slice' : ['$scores', 2, 1]}}},
                               {'$project' : {'name' : 1, 'examscore' : {'$min' : '$examcolumn.score'},
                                              'quizscore' : {'$min' : '$quizcolumn.score'},
                                              'homeworkscore' : {'$min' : '$homeworkcolumn.score'}}},
                               {'$project' : {'name' : 1, 'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1,
                                                          'Examresult' : {'$cond' : [{'$gt': ['$examscore', 40]}, 1, 0]},
                                                          'Quizresult' : {'$cond' : [{'$gt': ['$quizscore', 40]}, 1, 0]},
                                                          'Homeworkresult' : {'$cond' : [{'$gt': ['$homeworkscore', 40]}, 1, 0]}}},
                               {'$project' : {'name' : 1, 'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1,
                                              'Sumofresult' : {'$sum' : ['$Examresult', '$Quizresult', '$Homeworkresult']}}},
                               {'$project' : {'name' : {'$cond' : [{'$eq': ['$Sumofresult', 0]}, '$name',0]},
                                             'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1}},
                               {'$match' : {'name' : {'$ne': 0}}}
                              ]):
    collection6.insert_one(i)

# 7) Create a new collectio which consists of students who scored above pass mark in all the categories.

In [15]:
for i in collection.aggregate([{'$project' : {'name' : 1, 'examcolumn' : {'$slice' : ['$scores', 1]},
                                              'quizcolumn' : {'$slice' : ['$scores', 1, 1]},
                                              'homeworkcolumn' : {'$slice' : ['$scores', 2, 1]}}},
                               {'$project' : {'name' : 1, 'examscore' : {'$min' : '$examcolumn.score'},
                                              'quizscore' : {'$min' : '$quizcolumn.score'},
                                              'homeworkscore' : {'$min' : '$homeworkcolumn.score'}}},
                               {'$project' : {'name' : 1, 'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1,
                                                          'Examresult' : {'$cond' : [{'$gt': ['$examscore', 40]}, 1, 0]},
                                                          'Quizresult' : {'$cond' : [{'$gt': ['$quizscore', 40]}, 1, 0]},
                                                          'Homeworkresult' : {'$cond' : [{'$gt': ['$homeworkscore', 40]}, 1, 0]}}},
                               {'$project' : {'name' : 1, 'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1,
                                              'Sumofresult' : {'$sum' : ['$Examresult', '$Quizresult', '$Homeworkresult']}}},
                               {'$project' : {'name' : {'$cond' : [{'$eq': ['$Sumofresult', 3]}, '$name',0]},
                                             'examscore' : 1, 'quizscore' : 1, 'homeworkscore' : 1}},
                               {'$match' : {'name' : {'$ne': 0}}}
                              ]):
    collection7.insert_one(i)